In [2]:
## https://www.youtube.com/watch?v=NoKvCREx36Q
import os
from posixpath import split 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
import tensorflow as tf
import pandas as pd
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import pickle

ds_train = tf.data.TextLineDataset("emotions_train.csv")
ds_test = tf.data.TextLineDataset("emotions_test.csv")

def filter_train(line):
    split_line = tf.strings.split(line, ",", maxsplit=1)
    sentiment_category = split_line[1] # 

    return (
        True
        if sentiment_category != None
        else False
    )

# for line in ds_train.skip(1).take(5):
#     print(tf.strings.split(line, ",", maxsplit=2))

tokenizer = tfds.deprecated.text.Tokenizer()

def build_vocabulary(ds_train, threshold=10):
    """ Build a vocabulary """
    frequencies = {}
    vocabulary = set()
    vocabulary.update(["sostoken"])
    vocabulary.update(["eostoken"])

    for line in ds_train.skip(1):
        split_line = tf.strings.split(line, ",", maxsplit=2)
        review = split_line[2]
        tokenized_text = tokenizer.tokenize(review.numpy().lower())

        for word in tokenized_text:
            if word not in frequencies:
                frequencies[word] = 1
            else:
                frequencies[word] += 1

            # if we've reached the threshold
            if frequencies[word] == threshold:
                vocabulary.update(tokenized_text)

    return vocabulary

# Build vocabulary and save it to vocabulary.obj
vocabulary = build_vocabulary(ds_train)
vocab_file = open("vocabulary.obj", "wb")
pickle.dump(vocabulary, vocab_file)

# Loading the vocabulary
# vocab_file = open("vocabulary.obj", "rb")
# vocabulary = pickle.load(vocab_file)

encoder = tfds.deprecated.text.TokenTextEncoder(
    list(vocabulary), oov_token="<UNK>", lowercase=True, tokenizer=tokenizer,
)

def my_encoder(text_tensor, label):
    encoded_text = encoder.encode(text_tensor.numpy())
    return encoded_text, label

def encode_map_fn(line):
    split_line = tf.strings.split(line, ",", maxsplit=4)
    label_str = split_line[1]  # neg, pos
    review = "sostoken " + split_line[2] + " eostoken"
    label = 0

    if label_str == "joy":
        label = 0
    elif label_str == "sadness":
        label = 1
    elif label_str == "fear":
        label = 2
    elif label_str == "anger":
        label = 3

    (encoded_text, label) = tf.py_function(
        my_encoder, inp=[review, label], Tout=(tf.int64, tf.int32),
    )

    encoded_text.set_shape([None])
    label.set_shape([])
    return encoded_text, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = ds_train.map(encode_map_fn, num_parallel_calls=AUTOTUNE).cache()
ds_train = ds_train.shuffle(25000)
ds_train = ds_train.padded_batch(32, padded_shapes=([None], ()))

ds_test = ds_test.map(encode_map_fn)
ds_test = ds_test.padded_batch(32, padded_shapes=([None], ()))

model = keras.Sequential(
    [
        layers.Masking(mask_value=0),
        layers.Embedding(input_dim=len(vocabulary) + 2, output_dim=32,),
        layers.GlobalAveragePooling1D(),
        layers.Dense(64, activation="relu"),
        layers.Dense(4, activation="softmax"),
    ]
)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(3e-4, clipnorm=1),
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=300, verbose=2)
model.evaluate(ds_test)


'''

'''



C:\Users\Ciner\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/300


C:\Users\Ciner\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


10/10 - 1s - loss: 1.3865 - accuracy: 0.2114 - 1s/epoch - 146ms/step
Epoch 2/300
10/10 - 0s - loss: 1.3855 - accuracy: 0.2871 - 9ms/epoch - 900us/step
Epoch 3/300
10/10 - 0s - loss: 1.3848 - accuracy: 0.3060 - 9ms/epoch - 900us/step
Epoch 4/300
10/10 - 0s - loss: 1.3839 - accuracy: 0.3281 - 9ms/epoch - 900us/step
Epoch 5/300
10/10 - 0s - loss: 1.3827 - accuracy: 0.3628 - 8ms/epoch - 800us/step
Epoch 6/300
10/10 - 0s - loss: 1.3817 - accuracy: 0.3596 - 9ms/epoch - 900us/step
Epoch 7/300
10/10 - 0s - loss: 1.3807 - accuracy: 0.3849 - 9ms/epoch - 900us/step
Epoch 8/300
10/10 - 0s - loss: 1.3789 - accuracy: 0.4101 - 10ms/epoch - 1ms/step
Epoch 9/300
10/10 - 0s - loss: 1.3771 - accuracy: 0.4353 - 10ms/epoch - 1ms/step
Epoch 10/300
10/10 - 0s - loss: 1.3755 - accuracy: 0.4953 - 9ms/epoch - 900us/step
Epoch 11/300
10/10 - 0s - loss: 1.3734 - accuracy: 0.4700 - 10ms/epoch - 1ms/step
Epoch 12/300
10/10 - 0s - loss: 1.3711 - accuracy: 0.5110 - 10ms/epoch - 1ms/step
Epoch 13/300
10/10 - 0s - loss

'\n\n'

In [7]:
test = tf.data.TextLineDataset("hope.csv")
test = test.map(encode_map_fn)
test = test.padded_batch(32, padded_shapes=([None], ()))


prediction = model.predict(test)
print("prediction shape: ", prediction)




prediction shape:  [[9.7137883e-02 4.4755888e-01 2.2566977e-01 2.2963345e-01]
 [3.9265715e-13 8.1258351e-09 9.9971992e-01 2.8008225e-04]]
